### 模型推理

In [1]:
# 导入相关包
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

In [7]:
# 定义推理方法
def predict(model, tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", padding = True, truncation=True)
    # print(inputs)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    # print(predictions[0])
    return predictions[0]

In [15]:
# 定义推理后数据处理方法
def process_result(result, text):
    # 记录语句中出现的实体信息
    entity_list = []
    # 定义出标签映射关系
    id2Lable = {0: 'O',1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}
    # 记录本次循环出现的实体标签
    label_list = []
    # 记录标签的key
    label_key_list = []
    # 初始化一个字典用于防止识别出来的实体
    entity_dict = {}
    for i in range(1, len(result) - 1):
        # print(i, end = '')
        if result[i].item() != 0:
            label = id2Lable[result[i].item()]
            # 针对第一个实体标签
            if len(label_list) == 0:
                # 记录标签
                label_list.append(label)
                label_key_list.append(result[i].item())
                # 将实体记录到字典中
                entity_dict['entity'] = label.split('-')[1]
                entity_dict['context'] = text[i-1]
                # 跳出本次循环
                continue
                
            # 判断label 是属于什么标签, 将其与记录list 中最后一个标签进行比较
            last_label = label_list[-1]
            if label == last_label:
                # 如果两个标签一前一后完全相同，分为两种情况
                # 1. 两个“B” 开头的两个实体
                if label.split('-')[0] == 'B':
                    # 将 entity_dict 放入entity_list 中并清空 entity_dict
                    entity_list.append(entity_dict.copy())
                    entity_dict.clear()
                    entity_dict['entity'] = label.split('-')[1]
                    entity_dict['context'] = text[i-1]
                    
                # 2. 两个“I” 开头的两个实体的一部分
                if label.split('-')[0] == 'I':
                    # 取出context 进行追加
                    entity_dict['context'] = entity_dict['context'] + text[i-1]
            else:
                # 如果标签不一样也分为三种情况
                # 1.同一个实体的开始标签和标记标签
                if label.split('-')[1] == last_label.split('-')[1]:
                    # 如果前后两个实体的类型一致, 后一个实体的开始key 小于前一个
                    if result[i].item() < label_key_list[-1]:
                        # 将 entity_dict 放入entity_list 中并清空 entity_dict
                        entity_list.append(entity_dict.copy())
                        entity_dict.clear()
                        entity_dict['entity'] = label.split('-')[1]
                        entity_dict['context'] = text[i-1] 
                    else:
                        # 不满足这个规律则说明当前实体是连续的
                        entity_dict['context'] = entity_dict['context'] + text[i-1]
                    
                # 2.两个不同实体的分界
                if label.split('-')[1] != last_label.split('-')[1]:
                    # 将 entity_dict 放入entity_list 中并清空 entity_dict
                    entity_list.append(entity_dict.copy())
                    entity_dict.clear()
                    entity_dict['entity'] = label.split('-')[1]
                    entity_dict['context'] = text[i-1]

            # 每遇到一个标签就将标签记录下来
            label_key_list.append(result[i].item())
            label_list.append(label)
    # 遍历结束，将最后一个实体放入列表中
    if (len(entity_dict) > 0):
        entity_list.append(entity_dict.copy())
    return entity_list

In [13]:
def model_predict(text):
    # 根基不同checkpoint的模型进行同样文本的推理
    checkpoint_value_list = ['500', '1000', '1500', '2000', '2155']
    # 定义模型路径模版
    model_path_pattern = '/kaggle/input/doushabao4766-ner-pretrained/ner_train/checkpoint-{value}/'
    
    for value in checkpoint_value_list:
        # 构造不同检查点模型加载路径
        model_path = model_path_pattern.format(value = value)
        # 加载微调模型
        model = AutoModelForTokenClassification.from_pretrained(model_path)
        # 加载分词器
        tokenizer = AutoTokenizer.from_pretrained(model_path)
    
        # 进行推理
        predict_result = predict(model, tokenizer, text)
        # 推理结果处理
        result = process_result(predict_result, text)
        print("当前检查点（{value}）推理结果如下：".format(value = value))
        print(result)
        print('\n')

In [16]:
# 开始使用不同checkpoint 模型进行推理
text = "广州市公安局天河区分局官方微博发布警情通报称张三违法犯罪"
model_predict(text)

当前检查点（500）推理结果如下：
[{'entity': 'ORG', 'context': '广州市公安局'}, {'entity': 'ORG', 'context': '天河区分局'}, {'entity': 'PER', 'context': '张三'}]


当前检查点（1000）推理结果如下：
[{'entity': 'ORG', 'context': '广州市公安局天河区分局'}, {'entity': 'PER', 'context': '张三'}]


当前检查点（1500）推理结果如下：
[{'entity': 'ORG', 'context': '广州市公安局天河区分局'}, {'entity': 'PER', 'context': '张三'}]


当前检查点（2000）推理结果如下：
[{'entity': 'ORG', 'context': '广州市公安局天河区分局'}, {'entity': 'PER', 'context': '张三'}]


当前检查点（2155）推理结果如下：
[{'entity': 'ORG', 'context': '广州市公安局天河区分局'}, {'entity': 'PER', 'context': '张三'}]




In [17]:
# 开始使用不同checkpoint 模型进行推理
text = "双方确定了今后发展中美关系的指导方针"
model_predict(text)

当前检查点（500）推理结果如下：
[{'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '美'}]


当前检查点（1000）推理结果如下：
[{'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '美'}]


当前检查点（1500）推理结果如下：
[{'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '美'}]


当前检查点（2000）推理结果如下：
[{'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '美'}]


当前检查点（2155）推理结果如下：
[{'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '美'}]




In [18]:
text = "陈斌华表示，经核查，“黑熊学院”在民进党当局和外部干涉势力扶持下，打着“讲座”、“培训”、“户外演练”甚至“亲子活动”等幌子，明目张胆培育“暴力台独分子”，公然从事“台独”分裂活动，是不折不扣的“台独”基地。沈伯洋作为“黑熊学院”负责人，有组织有计划地从事“台独”分裂活动，大肆宣扬“台独”分裂谬论，恶意向台湾民众特别是青少年兜售“台独”主张和“仇中”思想，广泛散播台海暴力冲突种子。曹兴诚为“黑熊学院”从事分裂国家犯罪活动提供巨额资金支持，卖力鼓吹“以武拒统”、“全民皆兵”，还投资策动拍摄“台独”题材影视剧，以各种方式灌输“反中谋独”的精神毒素。沈、曹“台独”思想顽固、涉“独”言论嚣张、谋“独”行径恶劣、肆意“倚外抗中”，赤裸裸地分裂国家、煽动分裂国家，严重危害台海和平稳定，严重损害两岸同胞共同利益和中华民族根本利益"

model_predict(text)

当前检查点（500）推理结果如下：
[{'entity': 'PER', 'context': '陈斌华'}, {'entity': 'ORG', 'context': '黑熊学院'}, {'entity': 'ORG', 'context': '民进党'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'PER', 'context': '沈伯洋'}, {'entity': 'ORG', 'context': '黑熊学院'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '台湾'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '台海'}, {'entity': 'PER', 'context': '曹兴诚'}, {'entity': 'ORG', 'context': '黑熊学院'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '中'}, {'entity': 'PER', 'context': '沈'}, {'entity': 'PER', 'context': '曹'}, {'entity': 'LOC', 'context': '台'}, {'entity': 'LOC', 'context': '中'}, {'entity': 'LOC', 'context': '台海'}, {'entity': 'LOC', 'context': '两岸'}, {'entity': 'LOC', 'context': '中华'}]


当前检查点（1000）推理结果如下：
[{'entity': 'PER', 'context': '陈斌华'}, {'entity': 'ORG', 'contex